# En este colab se evalúa el accuracy del modelo entrenado por BSC, sobre el test set de XNLI 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MAF-ProyectoNLP/Validacion-Modelos/blob/main/BNE_TE%20evaluacion%20XNLI.ipynb)

*0*. Se instalan las bibliotecas necesarias

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 12.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.1 MB/s eta 0:00:00


*1*. Se importan las dependencias, y se definen valores de variables

In [ ]:
import torch.nn as nn
import torch
from transformers import RobertaForSequenceClassification, RobertaConfig, RobertaTokenizer
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import datasets
import numpy as np
from psutil import virtual_memory
from sklearn.metrics import roc_auc_score, accuracy_score

from datasets import load_dataset

torch.manual_seed(555)

tam_val = 8

L_RATE = 1e-5
MAX_LEN = 256

BATCH_SIZE = 8
NUM_CORES=1
L_RATE = 1e-5
MAX_LEN = 256

NUM_EPOCHS = 3

print(torch.__version__)

1.13.1+cu116


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Feb 11 12:41:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    49W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
device

device(type='cuda')

*2*. Definimos y cargamos el modelo a utilizar, en este caso el modelo finetuneado de BNE con text enconding

In [ ]:
MODEL_TYPE = "PlanTL-GOB-ES/roberta-large-bne-te"

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_TYPE)
model =  RobertaForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels=3)

model.to(device)

*3*.CompDataset es la clase utilizada para la tokenización de la frases

In [ ]:
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,   
                    add_special_tokens = True,  
                    max_length = MAX_LEN,        
                    pad_to_max_length = True,
                    return_attention_mask = True,
                    return_tensors = 'pt', 
                    padding="max_length", 
                    truncation=True
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        target = torch.tensor(self.df_data.loc[index, 'label'])

        sample = (padded_token_list, att_mask, target)
        return sample

    def __len__(self):
        return len(self.df_data) 

*4*. Se carga el dataset de XNLI

In [ ]:
dataset = load_dataset("xnli","es")
print(dataset["test"][100])
val_ds=dataset["test"]
df_val = pd.DataFrame(val_ds)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Dataset xnli downloaded and prepared to /root/.cache/huggingface/datasets/xnli/es/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'premise': 'Dijeron: Estamos pagando por un lugar para que te quedes.', 'hypothesis': 'Están pagando por la vivienda para mí y mis hermanos.', 'label': 1}


In [ ]:
test_data = CompDataset(df_val)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                       num_workers=NUM_CORES)

*5*. Se define la función para calcular el accuracy por batch

In [ ]:
def calculate_accuracy(test_dataloader):
  total = 0

  for j, batch in enumerate(test_dataloader):
      
    val_status = 'Batch ' + str(j) + ' of ' + str(len(test_dataloader))
    
    print(val_status)

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)    

    import os
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

    outputs = model(b_input_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

    preds = outputs[1].detach().cpu().numpy()

    y_true = b_labels.detach().cpu().numpy()
    y_pred = np.argmax(preds, axis=1)

    val_acc = accuracy_score(y_true, y_pred)
    print('Val acc: ', val_acc)
    total+=val_acc

  prom_acc=total/len(test_dataloader)  
  
  return prom_acc

In [ ]:
print( "Accuracy del Test set: "+str(calculate_accuracy(test_dataloader)))

Batch 0 of 627
Val acc:  1.0
Batch 1 of 627
Val acc:  1.0
Batch 2 of 627
Val acc:  1.0
Batch 3 of 627
Val acc:  0.875
Batch 4 of 627
Val acc:  0.875
Batch 5 of 627
Val acc:  0.625
Batch 6 of 627
Val acc:  1.0
Batch 7 of 627
Val acc:  0.875
Batch 8 of 627
Val acc:  0.875
Batch 9 of 627
Val acc:  0.875
Batch 10 of 627
Val acc:  0.625
Batch 11 of 627
Val acc:  0.625
Batch 12 of 627
Val acc:  0.75
Batch 13 of 627
Val acc:  0.875
Batch 14 of 627
Val acc:  0.875
Batch 15 of 627
Val acc:  0.75
Batch 16 of 627
Val acc:  0.875
Batch 17 of 627
Val acc:  0.875
Batch 18 of 627
Val acc:  0.875
Batch 19 of 627
Val acc:  0.5
Batch 20 of 627
Val acc:  1.0
Batch 21 of 627
Val acc:  1.0
Batch 22 of 627
Val acc:  0.75
Batch 23 of 627
Val acc:  1.0
Batch 24 of 627
Val acc:  0.875
Batch 25 of 627
Val acc:  0.875
Batch 26 of 627
Val acc:  0.875
Batch 27 of 627
Val acc:  1.0
Batch 28 of 627
Val acc:  0.5
Batch 29 of 627
Val acc:  1.0
Batch 30 of 627
Val acc:  0.875
Batch 31 of 627
Val acc:  0.875
Batch 32 of